# LEGALST-123 Folium Heatmaps Lab
## I did not do a great job of this one because of needing to group the coordinates by time unit

---

In this lab, students will learn how to construct a heatmap, as well as an interactive heat map. This will also be a component of the take-home problem set. This builds ontop of the folium labs from last week.


In [1]:
# dependencies
# from datascience import *
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import folium
import json
import os

In [2]:
!ls

Folium_Heatmaps_jon.ipynb Heatmaps_solutions.ipynb
Heatmaps.ipynb            data


In [3]:
!pip install folium --upgrade --user
import folium.plugins # The Folium Javascript Map Library
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

Requirement already up-to-date: folium in /Users/jmarshall/anaconda3/lib/python3.6/site-packages (0.8.0)


---

## The Data <a id='data'></a>
---

Today we'll be working with data on Berkeley crime calls, courtesy of the Berkeley Police department. Take a look at the metadata [here.](https://data.cityofberkeley.info/Public-Safety/Berkeley-PD-Calls-for-Service/k2nh-s5h5)

Note: this data set has already undergone a fair amount of cleaning to format it for our purposes (e.g. extracting the longitude and latitude, removing null values, and dropping irrelevant columns). You can see the original data at the source website.

Then, run the cell below to load the data into a Table.  

### need to get out of Tables and into Pandas syntax

In [4]:
calls = pd.read_csv('data/berkeley_crime_0218.csv', index_col=0)
calls.head(5)

,OFFENSE,CVLEGEND,BLKADDR,City,State,Day,Lat,Lon,timestamp
CASENO,,,,,,,,,
17076632,BURGLARY AUTO,BURGLARY - VEHICLE,1300 SAN PABLO AVE,Berkeley,CA,Monday,37.880262,-122.295809,2017-12-18 09:45:00
17092227,THEFT MISD. (UNDER $950),LARCENY,1600 SHATTUCK AVE,Berkeley,CA,Monday,37.878112,-122.269114,2017-10-30 10:25:00
18004102,BURGLARY COMMERCIAL,BURGLARY - COMMERCIAL,1400 SHATTUCK AVE,Berkeley,CA,Saturday,37.881957,-122.269551,2018-01-20 05:23:00
17065730,ALCOHOL OFFENSE,LIQUOR LAW VIOLATION,SOLANO AVENUE & COLUSA AVE,Berkeley,CA,Saturday,37.891368,-122.279257,2017-10-28 20:08:00
18000630,BURGLARY AUTO,BURGLARY - VEHICLE,800 POTTER ST,Berkeley,CA,Wednesday,37.851255,-122.292509,2018-01-03 19:00:00


When working with any new data set, it's a good idea to get to know it first. Use the following cell and the information on cityofberkeley.org to answer some basic questions:
- What information does this table contain? What are the different columns?
- How large is the data set? 
- What kinds of questions could we answer using this data set?

In [5]:
# what are dimensions of dataframe
print('shape of dataframe as rows, columns is ',calls.shape)
# what are the columns
print('variables: ', calls.columns)

shape of dataframe as rows, columns is  (5030, 9)
variables:  Index(['OFFENSE', 'CVLEGEND', 'BLKADDR', 'City', 'State', 'Day', 'Lat', 'Lon',
       'timestamp'],
      dtype='object')


## Heatmap <a id='data'></a>

Let's see if we can figure out what heatmap does and why it is useful.But first, we're going to quickly review how to use folium.Map. Again, you should consult the [python visualizer](https://python-visualization.github.io/folium/quickstart.html) for a refresher in case you forget how folium works!

Plot a map of the United States again using folium.Map.

<b>Reminder</b>: It is in the order of lat, log and the larger the zoom_start is the closer the map is.

In [6]:
# First, we create a folium Map
example_map1 = folium.Map([39.83, -98.59], zoom_start=6, tiles='Stamen Toner')
example_map1

### Key Note <br> _(we need to de- Table this part as well)_

Heatmaps do not take Tables so you will need to provide a list of lat, lons, i.e. a list of lists. 

Imagine that it looks something like this: `[[lat, lon],[lat, lon],[lat, lon],[lat, lon],[lat, lon]]`. This means if you were given a Table, there are a few steps you'd have to take.

1. Make sure the lat and lon are floats.
2. Filter the Table for the correct rows and columns.

What is something else you believe you'll need to check for to make sure that Heatmap will work?

**check for missing data!**

Our data set today has already had the NaNs filtered out, but that might not be true for data you work with in the future...

Run the next cell to generate a set of dummy `[[lat, lon]]` pairs for the HeatMap. Don't worry about the information itself. Instead, note how the array is formatted.

In [7]:
# The first two lines generate an array of small random numbers.
# The third line adds the random numbers to the pair [48, 5] to get 100 latitude, longitude pairs near [48, 5]
data = (np.random.normal(size=(100, 2)) *
        np.array([[1, 1]]) +
        np.array([[39.83, -98.59]])).tolist()
data

[[39.61116852570612, -99.3730290265691],
 [39.061436710837185, -98.22883201985954],
 [40.17516783572528, -99.43868187659417],
 [39.366858597618524, -100.00041565466648],
 [38.75704639135433, -97.204841493212],
 [39.575962904235425, -99.21715420926272],
 [39.122167719051454, -98.18936821932247],
 [38.746442581406214, -98.29730763314366],
 [39.727699870678805, -97.71288545441593],
 [38.74033791378362, -98.43314851847698],
 [39.3725381346995, -98.99724480066111],
 [40.88350115854812, -97.64126866014324],
 [40.174817650847764, -99.54957049776203],
 [40.6762388444551, -98.34553602830671],
 [42.337426880815535, -99.0734748755149],
 [40.40723280494323, -100.41352008481378],
 [39.4482810355298, -99.04877351804919],
 [40.90570831455803, -97.22027170761469],
 [39.43533251883142, -99.10289110101289],
 [39.23780176188601, -99.31185448366598],
 [39.67870901137665, -98.17508146873409],
 [39.23494130588545, -97.04751265453027],
 [39.51771984807598, -98.16521007635296],
 [41.50675630972964, -95.976729

Then we can plot it on the map! The function is pretty simple: 
1. Create a Heatmap using the function `Heatmap(your_lat_lon_data)`
2. Add that Heatmap to your existing map with `add_to(your_map)`

In [8]:
# Add the HeatMap to the map
HeatMap(data).add_to(example_map1)

example_map1

Play around with your new Heatmap. What is it plotting? What kinds of things would a Heatmap be useful for?

**This heatmap is plotting a set of points normally distributed around the geographic center of the 

### Try It Out

Now, try making your own Heatmap using the Berkeley PD call data. First, plot a Folium Map of the Bay Area, just like you did last week.

In [9]:
#Plot the map of Berkeley
berk_coords = [37.8716, -122.2727]
berk_map = folium.Map(berk_coords, zoom_start=13, tiles='Stamen Toner')
berk_map

Next, extract your latitude and longitude data from the `calls` Table and save each to the variables `lat` and `lon`. We want the data as a numpy array, so don't use the `select` function; instead, index the Table by the correct column (e.g. `calls["Column_I_Want"]`).

In [10]:
lat = calls['Lat']
lon = calls['Lon']
lat

CASENO
17076632    37.880262
17092227    37.878112
18004102    37.881957
17065730    37.891368
18000630    37.851255
17064405    37.879348
17047894    37.873687
17092229    37.886740
17072806    37.887747
17055485    37.868489
17092083    37.870311
18090224    37.897129
17064647    37.871167
17062686    37.870396
17092385    37.867668
17059712    37.865772
17052078    37.880227
18004326    37.866936
17092540    37.858530
17092260    37.869279
17068151    37.858518
17092271    37.868785
17055890    37.864535
17055716    37.878722
17092472    37.851750
18006469    37.859006
17067957    37.854732
18005755    37.867852
17049622    37.866592
17051393    37.865009
              ...    
17066579    37.868667
17060562    37.884270
17091716    37.870121
18002944    37.867871
17074467    37.871743
17064746    37.861265
17092186    37.870652
17091981    37.862085
17066442    37.873921
17071366    37.866327
17069070    37.867176
17047720    37.865009
17092407    37.877378
17050590    37.867482
170

We have the right data, but it isn't in the right shape: we want an array of arrays, where the first column is latitudes, the second column is longitudes, and each row is a `[lat, lon]` pair (see the example above). We can do this by:
1. **Stacking** the `lat` array on top of the `lon` array into one larger array with `np.vstack`
2. **Transposing** our stacked array so the latitude and longitude are vertical columns, not horizontal rows.

Hint 1: the stacking function call looks something like `np.vstack((top_array, bottom_array))`
Hint 2: you can transpose an array by calling `.transpose()` on the array

In [11]:
# it seems like since we are using Pandas we could just do this from the original dataframe
call_locs = np.vstack((lat, lon)).transpose()
call_locs[:4]

array([[  37.880262, -122.295809],
       [  37.878112, -122.269114],
       [  37.881957, -122.269551],
       [  37.891368, -122.279257]])

Now, you have everything you need to make your HeatMap! Do so in the cell below.

In [12]:
#Create a Heatmap with the call data.
heatmap = HeatMap(call_locs).add_to(berk_map)

# Add it to your Berkeley map.
berk_map

What conclusions can you draw from this Heatmap?

**The heatmap shows a bunch of things. First, the calls seem to be recorded by intersection location, more or less. Second, the calls are concentrated along the main streets, especially south of campus, and at major intersections along University and San Pablo. The more residential parts of Berkeley both north and south are pretty quiet, except for a few hot spots (like California at Derby-Ward or thereabouts). Third, there are very few calls in North Berkeley and the Hills, except at Solano and Colusa and Marin and Euclid--traffic calls?**

## HeatMapwithTime <a id='data'></a>

Now what do you think is different with HeatMapwithTime?

**I suppose it shows some kind of animation that shows frequency with color changes. Fancy!**

In this example, we'll again use dummy data to show how it works. It follows a similar process to HeatMap. First, create another Folium Map centered at the geographical center of the USA.<br><br>
## We don't really need this second USA example map since the example heatmap with time is of Western Europe

In [13]:
# Create a folium Map at the USA's center
example_map2 = folium.Map([39.83, -98.59], zoom_start=6, tiles='Stamen Toner')
example_map2

Next, we will create more dummy location data to simulate locations associated with different dates. Don't worry too much about the code here, but you do need to understand how the output is shaped and why it needs to be shaped like that.

In [14]:
# This cell builds together an array of initial data to display on our HeatMapwithTime. Just as before, these are dummy 
# variables that are 100 copies of the center of the USA meant to simulate different locations in the area.
# Again, we have to use lon and lat in addition to time.
np.random.seed(3141592)
initial_data = (
    np.random.normal(size=(100, 2)) * np.array([[1, 1]]) +
    np.array([[48, 5]])
)

# Create even more random lat/lon pairs and group into 100 lists
# You don't need to know how to write this code
move_data = np.random.normal(size=(100, 2)) * 0.01

data = [(initial_data + move_data * i).tolist() for i in range(100)]
data[1]

[[47.52791226223265, 4.286855296250705],
 [47.70515318785451, 4.254559938968417],
 [46.572343290775486, 3.16589036807419],
 [48.517652364172655, 3.565009343464859],
 [47.9151469875597, 4.561948133015522],
 [47.73009213894775, 6.400078703194206],
 [49.26936459309292, 3.5699714093014285],
 [47.851397175019756, 3.8319427791390397],
 [48.04477172261817, 6.316007103139816],
 [46.479416288127055, 4.682820499868778],
 [48.24059429677636, 4.343909573828612],
 [49.09604239901807, 5.094088143933742],
 [48.28316532514145, 5.662784381971345],
 [48.12325977596014, 5.782001071996817],
 [46.44347099963196, 4.770453864106374],
 [49.29117678153405, 6.136711330741487],
 [47.00836652976502, 6.286928736906839],
 [46.86855706102264, 5.113019811657515],
 [48.612612228434514, 4.677966117580723],
 [48.84324127910088, 4.533402809342304],
 [49.816563455994675, 3.759361045283218],
 [47.402859931934984, 5.299758773151624],
 [47.59727798048993, 4.887621415574294],
 [47.96991918344748, 5.560256321942765],
 [47.5909

Since we're using HeatMapWithTime, we need an extra parameter: the dates for each list of lat/lon pairs. Run the next cell to create one.

In [15]:
# Generate a set of dates for this dummy data.
# Luckily for us, when you test this out for yourself, dates came with your data set.
# You don't need to write out this code, but do look it over and see if you can understand it.
from datetime import datetime, timedelta

time_index = [
    (datetime.now() + k * timedelta(1)).strftime('%Y-%m-%d') for
    k in range(len(data))
]
print('first 5 elements of time_index: ', time_index[:4])
type(time_index[1])

first 5 elements of time_index:  ['2019-02-25', '2019-02-26', '2019-02-27', '2019-02-28']


str

Finally, create the HeatMapWithTime by calling the constructor function on the data and settng the index to the set of dates you generated. Then, add it to your Map.

In [16]:
# This is the code on how to run HeatMapwithTime. Looks similar to code we saw above right?
m = folium.Map([48., 5.], zoom_start=6)

hm = HeatMapWithTime(
    data,
    index=time_index,
    auto_play=True,
)

hm.add_to(m)

m

Now try for yourself using the Berkeley `calls` data set.

## _The following needs to be taken out of Tables and put into Pandas_

_The first step is to get the data into the correct format. Create a new Table with two columns: Date, containing the data in the calls "timestamp" column, and Location, containing the call location data you used to make your HeatMap (the stacked and transposed latitudes and longitudes)._

*Hint: check your 1-18 lab, or the Datascience Table documentation for [creating](http://data8.org/datascience/_autosummary/datascience.tables.Table.with_columns.html#datascience.tables.Table.with_columns) and [grouping](http://data8.org/datascience/_autosummary/datascience.tables.Table.group.html) Tables. You're going to want to call `group` with the `list` function as the aggregator.*


In [17]:
# Create a new dataframe with the locations and timestamps of calls, grouped by date

locs_and_dates = calls.loc[:,['Lat', 'Lon', 'timestamp']]
dti = pd.to_datetime(locs_and_dates['timestamp'])
locs_and_dates['dti'] = dti

locs_and_dates.head()

,Lat,Lon,timestamp,dti
CASENO,,,,
17076632,37.880262,-122.295809,2017-12-18 09:45:00,2017-12-18 09:45:00
17092227,37.878112,-122.269114,2017-10-30 10:25:00,2017-10-30 10:25:00
18004102,37.881957,-122.269551,2018-01-20 05:23:00,2018-01-20 05:23:00
17065730,37.891368,-122.279257,2017-10-28 20:08:00,2017-10-28 20:08:00
18000630,37.851255,-122.292509,2018-01-03 19:00:00,2018-01-03 19:00:00


In [18]:
# what data type is the timestamp?
print(type(locs_and_dates.loc[17076632,'dti']))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [32]:
# now group the locations and timestamps by timestamp
# in pandas, what we really want to do is sort the values by datetime inplace=True
# but we want to .group_values not sort them, so that they all appear at same time
# and .groupby needs an aggregator function but it can only do that on one value not both lat & lon
locs_dates_grouped = locs_and_dates.groupby(['timestamp']).apply
locs_dates_grouped.head()

AttributeError: 'DataFrameGroupBy' object has no attribute 'list'

## This needs to be Pandas-ized <br>
Next, extract the dates and the grouped locations into two variables to put in your HeatMapWithTime. Note:

* HeatMapWithTime needs lists, so you'll need to convert your dates to a list using `.tolist()`
* The Table Group function converts everthing to arrays, and each array needs to be converted to a list. This is super annoying, so we've given you the code to do it. Just extract the grouped locations from the correct column and put the extracted data in the ellipses on the second line.

In [20]:
# I think we can skip this step by using what is in the dataframe with the .tolist() method on Series data
# berk_dates = ...
# berk_loc_by_date = [[x.tolist() for x in y] for y in ...]

Finally, create a Folium map of Berkeley, then create a [HeatMapwithTime](https://python-visualization.github.io/folium/docs-v0.5.0/plugins.html) and add it to your Berkeley map. The call looks like `HeatMapWithTime(<grouped locations>, index=<dates>`). Click the link for more documentation. And, try adding the argument `autoplay=True`.

In [21]:
#Plot the heatmap of Berkeley crime
berk_coords = [37.8716, -122.2727]
berk_map2 = folium.Map(berk_coords, zoom_start=13, tiles='OpenStreetmap')

# create a bunch of lists from the df Series data to pass to Heat Map With Time
lat_list = locs_dates_grouped['Lat'].tolist()
lon_list = locs_dates_grouped['Lon'].tolist()
time_index = locs_dates_grouped['dti'].tolist()
print('first five lines of time_index: ', time_index[:4])
time_index2 = [time_index[k].strftime('%Y-%m-%d') for       # get rid of  %H:%M:%S for now
    k in range(len(lat_list))
]

# take latitude and longitude and make it into a list of [lat, lon] pairs
data = np.vstack((lat_list, lon_list)).transpose().tolist()
print('data is of type ', type(data))
print('first five lines of coordinate data: ', data[:4])
print('first five lines of time_index2: ', time_index2[:4])
print('time_index2 type: ', type(time_index2[1]))
print('length of data list: ', len(data), ' length of time_index2 list: ', len(time_index2))


#finally, pass all the lists to HeatMapWithTime and then add that to basemap
hmwt_berk = HeatMapWithTime(
    data,
    index=time_index2,
    auto_play=True,
    max_speed=100
)
hmwt_berk.add_to(berk_map2)

berk_map2

AttributeError: Cannot access callable attribute 'tolist' of 'SeriesGroupBy' objects, try using the 'apply' method

What conclusions can you draw from this Heatmap?